In [57]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.image as mpimg
import os
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.models import save_model, load_model
from matplotlib import pyplot as plt

from pathlib import Path

In [31]:

batch_size = 128
image_size = (224, 224)
seed = 42
data_augmentation = True

In [3]:
data = pd.read_csv('archive/birds.csv')
data

,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
...,...,...,...,...,...
89880,524.0,valid/BLACK BREASTED PUFFBIRD/3.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89881,524.0,valid/BLACK BREASTED PUFFBIRD/4.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89882,524.0,valid/BLACK BREASTED PUFFBIRD/1.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89883,524.0,valid/BLACK BREASTED PUFFBIRD/2.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS


In [6]:
bird = pd.read_json('aves.birds.json')
bird_name = bird['bird_name'].tolist()

In [12]:
df_filtered = data[data['labels'].isin(bird_name)]
df_filtered

,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
...,...,...,...,...,...
87755,99.0,valid/BLUE MALKOHA/1.jpg,BLUE MALKOHA,valid,CEUTHMOCHARES AEREUS
87756,99.0,valid/BLUE MALKOHA/2.jpg,BLUE MALKOHA,valid,CEUTHMOCHARES AEREUS
87757,99.0,valid/BLUE MALKOHA/3.jpg,BLUE MALKOHA,valid,CEUTHMOCHARES AEREUS
87758,99.0,valid/BLUE MALKOHA/4.jpg,BLUE MALKOHA,valid,CEUTHMOCHARES AEREUS


In [13]:
df_filtered.loc[:, 'filepaths'] = df_filtered['filepaths'].apply(lambda x: f"archive/{x}")

In [14]:
df_filtered

,class id,filepaths,labels,data set,scientific name
0,0.0,archive/train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,archive/train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,archive/train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,archive/train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,archive/train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
...,...,...,...,...,...
87755,99.0,archive/valid/BLUE MALKOHA/1.jpg,BLUE MALKOHA,valid,CEUTHMOCHARES AEREUS
87756,99.0,archive/valid/BLUE MALKOHA/2.jpg,BLUE MALKOHA,valid,CEUTHMOCHARES AEREUS
87757,99.0,archive/valid/BLUE MALKOHA/3.jpg,BLUE MALKOHA,valid,CEUTHMOCHARES AEREUS
87758,99.0,archive/valid/BLUE MALKOHA/4.jpg,BLUE MALKOHA,valid,CEUTHMOCHARES AEREUS


In [18]:
train_df = df_filtered[df_filtered['data set'] == 'train']
test_df = df_filtered[df_filtered['data set'] == 'test']
valid_df = df_filtered[df_filtered['data set'] == 'valid']


In [22]:
if data_augmentation:
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        zoom_range=0.15,
        horizontal_flip=True
    )
else:
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )

valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [33]:
train_df = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepaths',
    y_col='labels',
    target_size=image_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

validation_df = valid_generator.flow_from_dataframe(
    dataframe=valid_df,
    x_col='filepaths',
    y_col='labels',
    target_size=image_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

test_df = valid_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepaths',
    y_col='labels',
    target_size=image_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

Found 15747 validated image filenames belonging to 100 classes.
Found 500 validated image filenames belonging to 100 classes.
Found 500 validated image filenames belonging to 100 classes.


In [41]:
def get_model():
    model = Sequential()

    # Base Model
    model.add(layers.Conv2D(filters=16, kernel_size=3, input_shape=(image_size[0], image_size[1], 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2, padding='same'))

    model.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2, padding='same'))
    
    model.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2, padding='same'))
    
    #Top Model
    model.add(layers.Flatten())
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(256))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(100, activation='softmax'))

    model.summary()
    
    return model

model = get_model()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 224, 224, 16)      448       
                                                                 
 batch_normalization_15 (Ba  (None, 224, 224, 16)      64        
 tchNormalization)                                               
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 112, 112, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 112, 112, 32)      4640      
                                                                 
 batch_normalization_16 (Ba  (None, 112, 112, 32)      128       
 tchNormalization)                                               
                                                      

In [42]:
checkpoint_path = "checkpoints"

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=4,
    restore_best_weights=True,
    verbose=0
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor="val_accuracy",
    save_best_only=True,
    verbose=0
)

In [43]:
lr = 0.0015
epochs = 15

In [44]:
model.compile(
    optimizer = tf.keras.optimizers.legacy.Adam(lr),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [45]:
H = model.fit(
    train_df,
    epochs=epochs,
    validation_data=validation_df,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/15
124/124 [==============================] - ETA: 0s - loss: 3.3075 - accuracy: 0.2298INFO:tensorflow:Assets written to: checkpoints/assets


INFO:tensorflow:Assets written to: checkpoints/assets


124/124 [==============================] - 166s 1s/step - loss: 3.3075 - accuracy: 0.2298 - val_loss: 6.5461 - val_accuracy: 0.0100
Epoch 2/15
124/124 [==============================] - ETA: 0s - loss: 2.3338 - accuracy: 0.4112INFO:tensorflow:Assets written to: checkpoints/assets


INFO:tensorflow:Assets written to: checkpoints/assets


124/124 [==============================] - 162s 1s/step - loss: 2.3338 - accuracy: 0.4112 - val_loss: 5.9460 - val_accuracy: 0.0140
Epoch 3/15
124/124 [==============================] - ETA: 0s - loss: 1.9597 - accuracy: 0.5003INFO:tensorflow:Assets written to: checkpoints/assets


...

124/124 [==============================] - 414s 3s/step - loss: 1.3718 - accuracy: 0.6300 - val_loss: 1.2837 - val_accuracy: 0.6700
Epoch 9/15
124/124 [==============================] - 412s 3s/step - loss: 1.3521 - accuracy: 0.6401 - val_loss: 1.3851 - val_accuracy: 0.6700
Epoch 10/15
124/124 [==============================] - ETA: 0s - loss: 1.3134 - accuracy: 0.6481INFO:tensorflow:Assets written to: checkpoints/assets


INFO:tensorflow:Assets written to: checkpoints/assets


124/124 [==============================] - 412s 3s/step - loss: 1.3134 - accuracy: 0.6481 - val_loss: 1.1116 - val_accuracy: 0.6960
Epoch 11/15
124/124 [==============================] - 413s 3s/step - loss: 1.1912 - accuracy: 0.6774 - val_loss: 1.2053 - val_accuracy: 0.6640
Epoch 12/15
124/124 [==============================] - 413s 3s/step - loss: 1.1963 - accuracy: 0.6745 - val_loss: 1.6121 - val_accuracy: 0.5940
Epoch 13/15
124/124 [==============================] - 418s 3s/step - loss: 1.1824 - accuracy: 0.6818 - val_loss: 1.1930 - val_accuracy: 0.6760
Epoch 14/15
124/124 [==============================] - ETA: 0s - loss: 1.0854 - accuracy: 0.7034INFO:tensorflow:Assets written to: checkpoints/assets


INFO:tensorflow:Assets written to: checkpoints/assets


124/124 [==============================] - 415s 3s/step - loss: 1.0854 - accuracy: 0.7034 - val_loss: 0.9167 - val_accuracy: 0.7480
Epoch 15/15
124/124 [==============================] - 412s 3s/step - loss: 1.0061 - accuracy: 0.7239 - val_loss: 2.4203 - val_accuracy: 0.5320


In [46]:
save_model(model, 'bird_classification_model.h5')

/var/folders/4c/f0gh85js6pdgrgj4rj7mkw000000gn/T/ipykernel_84213/3975298713.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'bird_classification_model.h5')


In [107]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('bird_classification_model.h5')

# Preprocess the input image
img_path = 'archive/valid/ASIAN GREEN BEE EATER/2.jpg'
img = image.load_img(img_path, target_size=image_size)  # Adjust height and width to match the model's input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize pixel values

# Make predictions
predictions = model.predict(img_array)

# Print the predicted class probabilities
class_list = valid_df['labels'].unique().tolist()
top_three_indices = np.argsort(predictions[0])[-3:][::-1]
top_three_birds = [(class_list[i], predictions[0][i]) for i in top_three_indices]

print("Top three predicted bird names and probabilities:")
for bird, probability in top_three_birds:
    print(f"{bird}: {probability:.4f}")

1/1 [==============================] - 0s 74ms/step
Top three predicted bird names and probabilities:
ASIAN GREEN BEE EATER: 0.9525
AFRICAN EMERALD CUCKOO: 0.0185
ALEXANDRINE PARAKEET: 0.0112
